## Dealing with Missing Data


### When reading in missing values, pandas will display them as NaN values.
### There are also newer specialized null pandas values such as pd.NaT to imply the value missing should be a timestamp.

Options for missing Data:
-  1. Keep them
-  2. Remove them
-   3. Replace them with a value
-   4. Replace them with a statistic (mean, median, mode)
-   5. Use interpolation to estimate the missing values
-   6. Use machine learning models to predict the missing values
-   7. Use domain knowledge to fill in the missing values

Keeping a missing data:
* Pros:
  - Easiest to do
  - Does not manipulate or change the true data
* Cons:
  - Many methods do not support NaN
  - Often there are reasonable guesses

Dropping or removing the missing data:(Makes sense when a lot of data is missing)
* Pros: 
  - Easy to do
  - Can do based on rules
* Cons:
  - Potential to lose a lot of dat or useful information
  - Limits trained models for future data.

(Often a good idea to calculate the percent of data which is missing)


Filling the missing data:
* Pros:
  - Potential to save a lot of data for use in training a model
* Cons:
  - Hardest to do and somewhat arbitrary
  - Potential to lead to false conclusions

* Filling the same value => good choice if NaN was a placeholder
* Filling with interpolated or estimated value => much harder and requires reasonable assumptions



In [25]:
import numpy as np
import pandas as pd

In [26]:
np.nan #old version of missing value
pd.NaT #newer version of missing value for timestamps
np.nan == np.nan #False, because NaN is not equal to itself
np.nan is np.nan #True, because NaN is the same object in memory

True

In [27]:
df = pd.read_csv('6_movie_scores.csv')

In [28]:
df

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
1,NaN,NaN,NaN,NaN,NaN,NaN
2,Hugh,Jackman,51.0,m,NaN,NaN
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [29]:
df.isnull() #returns a DataFrame of the same shape as df, with True for missing values and False for non-missing values

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,False,False,False,False,False,False
1,True,True,True,True,True,True
2,False,False,False,False,True,True
3,False,False,False,False,False,False
4,False,False,False,False,False,False


In [30]:
df['pre_movie_score'].notnull() #returns a Series of the same shape as df['pre_movie_score'], with True for non-missing values and False for missing values

0     True
1    False
2    False
3     True
4     True
Name: pre_movie_score, dtype: bool

In [31]:
df[(df['pre_movie_score'].isnull()) & (df['first_name'].notnull())] #returns a DataFrame with rows where pre_movie_score is missing and first_name is not missing

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
2,Hugh,Jackman,51.0,m,NaN,NaN


## Drop Data

In [32]:
df.dropna() #drops all rows with any missing values

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [33]:
df.dropna(thresh=1) #drops rows with less than 1 non-missing value

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
2,Hugh,Jackman,51.0,m,NaN,NaN
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [34]:
df.dropna(axis=1) #drops all columns with any missing values

""
0
1
2
3
4


In [35]:
df.dropna(axis=0) #drops all rows with any missing values (same as df.dropna())

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [36]:
df.dropna(subset=['last_name']) #drops rows where last_name is missing, but keeps other columns with missing values

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
2,Hugh,Jackman,51.0,m,NaN,NaN
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [ ]:
df.fullna() #fills all missing values with 0
df.fillna('New Value') #fills all missing values with 'New Value'

In [38]:
df['pre_movie_score'] = df['pre_movie_score'].fillna(0) #filling missing values in a specific column with 0
df

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.0,m,8.0,10.0
1,NaN,NaN,NaN,NaN,0.0,NaN
2,Hugh,Jackman,51.0,m,0.0,NaN
3,Oprah,Winfrey,66.0,f,6.0,8.0
4,Emma,Stone,31.0,f,7.0,9.0


In [39]:
df['pre_movie_score'].fillna(df['pre_movie_score'].mean()) #fills missing values in pre_movie_score with the mean of pre_movie_score

0    8.0
1    0.0
2    0.0
3    6.0
4    7.0
Name: pre_movie_score, dtype: float64

In [40]:
df.fillna(df.mean(numeric_only=True)) #fills missing values in numeric columns with their mean

,first_name,last_name,age,sex,pre_movie_score,post_movie_score
0,Tom,Hanks,63.00,m,8.0,10.0
1,NaN,NaN,52.75,NaN,0.0,9.0
2,Hugh,Jackman,51.00,m,0.0,9.0
3,Oprah,Winfrey,66.00,f,6.0,8.0
4,Emma,Stone,31.00,f,7.0,9.0


In [41]:
airline_tix = {'first':100,'business':np.nan,'economy-plus':50,'economy':30}

In [42]:
ser = pd.Series(airline_tix)

In [44]:
ser.interpolate() #interpolates missing values in the Series using linear interpolation

first           100.0
business         75.0
economy-plus     50.0
economy          30.0
dtype: float64